In [0]:
import sys
import os

In [0]:
# Go two levels up to reach the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))

if project_root not in sys.path:
    sys.path.append(project_root)

from modules.utils.date_utils import get_month_start_n_months_ago
from pyspark.sql.functions import date_format

In [0]:
# Get the first day of the month two months ago
two_months_ago_start = get_month_start_n_months_ago(2)

In [0]:
# Read the 'yellow_trips_enriched' table from the 'nyctaxi.02_silver' schema
# and filter to only include trips with a pickup datetime
# later than the start date from two months ago

df = spark.read.table("nyctaxi.02_silver.yellow_trips_enriched").filter(f"tpep_pickup_datetime > '{two_months_ago_start}'")

In [0]:
# Add a year_month column, formatted as yyyy-MM

df = df.withColumn("year_month", date_format("tpep_pickup_datetime", "yyyy-MM"))

In [0]:
# Write the yellow_trips data in JSON format to the External Table "yellow_trips_export"

df.write.\
    option("path", "abfss://nyctaxi-yellow@nyctaxistorage999000.dfs.core.windows.net/yellow_trips_export/").\
    format("json").\
    mode("append").\
    partitionBy("vendor", "year_month").\
    saveAsTable("nyctaxi.04_export.yellow_trips_export")